In [1]:
import argparse

def writeFile(fname, code):
	try:
		with open(fname, 'wb') as f:
			f.write(''.join(code))
	except IOError:
		exit('No such file or directory ' + fname)

def readFile(fname):
	try:
		with open(fname, 'rb') as f:
			text = f.read()
	except IOError:
		exit('No such file or directory ' + fname)
	return text

def getArgs():
	parser = argparse.ArgumentParser()
	parser.add_argument('inFile')
	parser.add_argument('outFile')
	return parser.parse_args()

 # Stream alignment and add message length
def alignment(msg):
	msg_len = len(msg) * 8
	msg.append(0x80)
	while len(msg)% 64 != 56:
		msg += [0]
	for i in range(8):
		msg.append(msg_len >> i * 8)
	return msg

In [2]:
import math

# 5 nonlinear bitwise functions
def F(j, x, y, z):
	if j < 16:
		return (x ^ y ^ z)%(2**32)
	if j < 32:
		return(x & y) | (~x & z)%(2**32)
	if j < 48:
		return ((x | ~y) ^ z)%(2**32)
	if j < 64:
		return (x & z) | (y & ~z)%(2**32)
	if j < 80:
		return (x ^ (y | ~z))%(2**32)

# Added hexadecimal constants
def K(j):
	if j < 16:
		return 0x00000000
	if j < 32:
		return 0x5A827999
	if j < 48:
		return 0x6ED9EBA1
	if j < 64:
		return 0x8F1BBCDC
	if j < 80:
		return 0xA953FD4E

def K1(j):
	if j < 16:
		return 0x50A28BE6
	if j < 32:
		return 0x5C4DD124
	if j < 48:
		return 0x6D703EF3
	if j < 64:
		return 0x7A6D76E9
	if j < 80:
		return 0x00000000

# циклический сдвиг влево на n бит
def rotateLeft(x, n):
	x = x & 0xFFFFFFFF
	return ((x << n) | (x >> (32-n))) & 0xFFFFFFFF

def rotateRight(x, n):
	x = x & 0xFFFFFFFF
	return ((x >> n) | (x << (32-n))) & 0xFFFFFFFF

def sub_bytes(i, start=0, end=0):
	i_str = hex(i)[2:]  				# skip 0x part
	i_sub = i_str[-end * 2: len(i_str) - start * 2]  # get the bytes we need
	return int(i_sub or '0', 16)  		# convert to back int

def slice_bytes(value, a=None, b=None, byteorder='little'):
	size = math.ceil(value.bit_length() / 8)
	value_bytes = value.to_bytes(size, byteorder)
	return int.from_bytes(value_bytes[a: b], byteorder)

def toLittleEndian(word):
	res = 0
	res |= ((word >> 0)  & 0xFF) << 24
	res |= ((word >> 8)  & 0xFF) << 16
	res |= ((word >> 16) & 0xFF) << 8
	res |= ((word >> 24) & 0xFF) << 0
	return res

In [ ]:
def rounds(buf, x):

	X     = []
	Y     = []
	dataX = []
	dataY = []

	QX = [0]*80
	QY = [0]*80
	FX = [0]*80
	FY = [0]*80

	A  = buf[0]		# 0x67452301		# H0
	B  = buf[1] 	# 0xefcdab89		# H1
	C  = buf[2] 	# 0x98badcfe		# H2
	D  = buf[3] 	# 0x10325476		# H3
	E  = buf[4] 	# 0xc3d2e1f0		# H4

	A1 = buf[0]
	B1 = buf[1]
	C1 = buf[2]
	D1 = buf[3]
	E1 = buf[4]

	# Determine constants for r (j) and rol (j)
	R = [
		0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,
		7, 4, 13, 1, 10, 6, 15, 3, 12, 0, 9, 5, 2, 14, 11, 8,
		3, 10, 14, 4, 9, 15, 8, 1, 2, 7, 0, 6, 13, 11, 5, 12,
		1, 9, 11, 10, 0, 8, 12, 4, 13, 3, 7, 15, 14, 5, 6, 2,
		4, 0, 5, 9, 7, 12, 2, 10, 14, 1, 3, 8, 11, 6, 15, 13
		]

	R1 = [
		5, 14, 7, 0, 9, 2, 11, 4, 13, 6, 15, 8, 1, 10, 3, 12,
		6, 11, 3, 7, 0, 13, 5, 10, 14, 15, 8, 12, 4, 9, 1, 2,
		15, 5, 1, 3, 7, 14, 6, 9, 11, 8, 12, 2, 10, 0, 4, 13,
		8, 6, 4, 1, 3, 11, 15, 0, 5, 12, 2, 13, 9, 7, 10, 14,
		12, 15, 10, 4, 1, 5, 8, 7, 6, 2, 13, 14, 0, 3, 9, 11
		]

	S = [
		11, 14, 15, 12, 5, 8, 7, 9, 11, 13, 14, 15, 6, 7, 9, 8,
		7, 6, 8, 13, 11, 9, 7, 15, 7, 12, 15, 9, 11, 7, 13, 12,
		11, 13, 6, 7, 14, 9, 13, 15, 14, 8, 13, 6, 5, 12, 7, 5,
		11, 12, 14, 15, 14, 15, 9, 8, 9, 14, 5, 6, 8, 6, 5, 12,
		9, 15, 5, 11, 6, 8, 13, 12, 5, 12, 13, 14, 11, 8, 5, 6
		]

	S1 = [
		8, 9, 9, 11, 13, 15, 15, 5, 7, 7, 8, 11, 14, 14, 12, 6,
		9, 13, 15, 7, 12, 8, 9, 11, 7, 7, 12, 7, 6, 15, 13, 11,
		9, 7, 15, 11, 8, 6, 6, 14, 12, 13, 5, 14, 13, 13, 7, 5,
		15, 5, 8, 11, 14, 14, 6, 14, 6, 9, 12, 9, 12, 5, 15, 8,
		8, 5, 12, 9, 12, 5, 14, 6, 8, 13, 6, 5, 15, 13, 11, 11
		]
	# The algorithm is executed along two parallel paths
	i = 0
	k = 1
	for j in range(80):

		T = A + F(j, B, C, D) + x[R[j]] + K(j)
		FX.insert(j,F(j, B, C, D))
		QX[j] = T
		QX.insert(j,T)

		T = (rotateLeft(T, S[j]) + E )%(2**32)
		# if(j == 0):
		#     print('L T1 : ',hex(T).upper())
		A = E 				# NEW Value A-E
		E = D
		D = rotateLeft(C, 10)
		C = B
		B = T
		#print('Type',type(T))
		X.insert(j,T)

		T =  A1 + F(79 - j, B1, C1, D1) + x[R1[j]] + K1(j)
		#print('QY%d %X'%(j,T))
		FY.insert(j,F(79 - j, B1, C1, D1))
		QY.insert(j,T)

		T  = (rotateLeft(T, S1[j]) + E1)%(2**32)
		A1 = E1 					# NEW Value A1 - E1
		E1 = D1
		D1 = rotateLeft(C1, 10)
		C1 = B1
		B1 = T
		Y.insert(j,T)

		#if(j == 0):
		#print('\nX[%d] %X, = X-4<<10 %X, FX %X, QX %X,    Y[%d] %X, = Y-4<<10 %X ,FY %X, QY %X, '%(j,X[j],A,FX[j],QX[j],j,Y[j],A1,FY[j],QY[j]) )
		#print('\nX[%d] %X  Y[%d] %X '%(j,X[j],j,Y[j]) )
		#print('%X' %(X[j]) ,type(X[j]) )
		dataX.append(X[j])
		dataY.append(Y[j])

	T      = (buf[1] + C + D1) %(2**32)
	buf[1] = (buf[2] + D + E1) %(2**32)
	buf[2] = (buf[3] + E + A1) %(2**32)
	buf[3] = (buf[4] + A + B1) %(2**32)
	buf[4] = (buf[0] + B + C1) %(2**32)
	buf[0] = T
	#print ('\nBuf :[{}]'.format(', '.join(hex(x).upper()[2:] for x in buf)))

	X.clear()
	Y.clear()

	return buf, dataX, dataY

In [ ]:
def calc_ripemd160(data):

	data = [ord(i) for i in data]  # returns the number representing the unicode code
	data = alignment(data)
	#print(data)

	# Buffer initialization
	buf    = [0] * 5
	buf[0] = 0x67452301		# H0
	buf[1] = 0xefcdab89		# H1
	buf[2] = 0x98badcfe		# H2
	buf[3] = 0x10325476		# H3
	buf[4] = 0xc3d2e1f0		# H4

	# we split the stream of bytes into a stream of words
	data_words = []

	for i in range(len(data) // 4):
		q = 0
		for j in range(4):
			q |= data[i * 4 + j] << j * 8
		data_words.append(q)

	#print("Data Words",len(data_words))
	#print('Data0 %X'%(data_words[0]) )
	# print('In : [{}]'.format(', '.join(hex(x) for x in data_words)))
	# print(type(data_words) ,type(data_words[0]))

	# Step 4 Evaluation in a loop
	# break the stream into blocks of 16 words
	for i in range(0, len(data_words), 16):
		x   = data_words[i:i+16]			# i block is entered in x
		buf,dataX,dataY = rounds(buf, x)
		#print('Round : ',x)
		#print ('[{}]'.format(', '.join(hex(x) for x in buf)))

	# Step 5 Calculation result
	out = ""
	for i in buf:
		out += "{:08X} ".format(toLittleEndian(i))

	#print ('Out : %X %s'%(data_words[0],res) )
	#print ('Out : %s\n'%(res) )
	return data_words, out, dataX, dataY

# main

In [ ]:
%%time
import pandas as pd

rowList    = []
#total     = 0x01000000  # จำนวนรอบทั้งหมด 16M 
total      = 0x1000  # จำนวนรอบทั้งหมด test
range_stop = total

for i in range(0x00000000, range_stop, 1):
    dataX  = []
    dataY  = []
    # สร้าง data1 โดยเปลี่ยนเวิร์ดแรกเท่านั้น
    word1 = f"{i:08X}"
    data1 = f"{word1} 00000000 00000000 00000000 00000000 00000000 00000000 00000000"
    data1 = data1.replace("-", "")
    ascii_string = ''
    x = 0
    y = 2
    l = len(data1)
    while y <= l:
        ascii_string += chr(int(data1[x:y], 16))
        x += 2
        y += 2

    data = ascii_string
    input, output, dataX, dataY = calc_ripemd160(data)

    dict1 = {'message': input, 'X': dataX, 'Y': dataY}
    rowList.append(dict1)

    # แสดงเปอร์เซ็นต์ความคืบหน้า
    if i % 1000 == 0 or i == total - 1:
        percent = (i + 1) * 100 / total
        print(f"\rProgress: {percent:.2f}%", end="")

   # Optional: Save every N rows to avoid memory issues
    if i % 0x7FFFF == 0 and i > 0:
        df = pd.DataFrame(rowList)
        df.to_csv(f'm0x0-part-{i//0x7FFFF:03d}.csv', index=False)
        rowList = []